In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Using the example from the lecture notes
txn_dict = {'10': 'apple,carrot,durian', 
            '20': 'banana,carrot,edamame',
            '30': 'apple,banana,carrot,edamame', 
            '40': 'banana,edamame'
            }

trans_df = pd.DataFrame.from_dict(txn_dict,orient='index')
trans_df = trans_df[0].str.split(',')

trans_df

10             [apple, carrot, durian]
20           [banana, carrot, edamame]
30    [apple, banana, carrot, edamame]
40                   [banana, edamame]
Name: 0, dtype: object

In [3]:
from apriori import *

In [4]:
def prune(data,supp):
    df = data[data.supp_count >= supp]
    return df

def count_itemset(transaction_df, itemsets):
    count_item = {}
    for item_set in itemsets:
        set_A = set(item_set)
        for row in transaction_df:
            set_B = set(row)

            if set_B.intersection(set_A) == set_A:
                if item_set in count_item.keys():
                    count_item[item_set] += 1

                else:
                    count_item[item_set] = 1

    data = pd.DataFrame()
    data['item_sets'] = count_item.keys()
    data['supp_count'] = count_item.values()
    # print("Candidate itemset table (Counting):\n", data)

    return data

def count_item(trans_items):
    count_ind_item = {}
    for row in trans_items:
        for i in range(len(row)):
            if row[i] in count_ind_item.keys():
                count_ind_item[row[i]] += 1
            else:
                count_ind_item[row[i]] = 1

    data = pd.DataFrame()
    data['item_sets'] = count_ind_item.keys()
    data['supp_count'] = count_ind_item.values()
    data = data.sort_values('item_sets')

    return data

def join(list_of_items):
    itemsets = []
    i = 1
    for entry in list_of_items:
        proceding_items = list_of_items[i:]
        for item in proceding_items:
            if(type(item) is str):
                if entry != item:
                    tuples = (entry, item)
                    itemsets.append(tuples)
            else:
                if entry[0:-1] == item[0:-1]:
                    tuples = entry+item[1:]
                    itemsets.append(tuples)
        i = i+1
    if(len(itemsets) == 0):
        return None

    return itemsets

def apriori(trans_data,supp=10):
    freq = pd.DataFrame()

    df = count_item(trans_data)

    while(len(df) != 0):

        df = prune(df, supp)
        # print("Minsup =", supp,"\n")
        # print("Freq itemset table (Pruned):\n", df)

        if len(df) > 1 or (len(df) == 1 and int(df.supp_count >= supp)):
            freq = df

        itemsets = join(df.item_sets)

        if(itemsets is None):
            return freq

        df = count_itemset(trans_data, itemsets)
        
    return freq

In [5]:
apriori(trans_data=trans_df, supp=3)

,item_sets,supp_count
1,"(banana, edamame)",3
